In [1]:
import pywt
import pandas as pd
import numpy as np
import os
import glob

In [44]:
# 将从OrCAD Pspice文件中读出来的数据整理后存储

# 加载CSV文件
file_path = "C://Users\Ceren\Desktop//test_12.csv"  # 请替换'你的文件路径'为实际的文件路径
df = pd.read_csv(file_path)

# 去除列名的前后空格
df.columns = df.columns.str.strip()

# 定义保存新文件的基础路径
base_file_path = 'E:\毕设\电路仿真\仿真结果//four_op_amp\原始数据文件//R6+/12_'  # 请替换'你的保存路径'为实际希望保存文件的路径

ii=0

# 遍历除了'Time'列之外的所有列，并将数据保存到新文件中
for column in df.columns[1:]:  # 跳过'Time'列
    # 定义当前列的新文件路径
    new_file_path = f'{base_file_path}{ii}.csv'
    ii=ii+1
    # 提取当前列以及'Time'列的数据
    data_to_save = df[['Time', column]].copy()
    
    # 将数据保存到CSV文件
    data_to_save.to_csv(new_file_path, index=False)

print("数据已成功保存到指定文件中。")

数据已成功保存到指定文件中。


In [70]:


def process_and_save_features(file_path, output_dir, wavelet='db2', max_level=None):
    # 读取数据，跳过第一行（列名）
    df = pd.read_csv(file_path, header=None, skiprows=1)
    
    # 将第二列转换为浮点数，用作信号
    signal = df.iloc[:, 1].astype(float).values
    
    # 执行小波包分析
    wp = pywt.WaveletPacket(data=signal, wavelet=wavelet, mode='symmetric', maxlevel=max_level)
    
    # 提取特征：这里我们简单地使用每个节点的能量作为特征
    features = {node.path: np.sum(node.data**2) for node in wp.get_level(max_level or wp.maxlevel, 'freq')}
    
    # 将特征保存到DataFrame中
    features_df = pd.DataFrame([features])
    
    # 确保输出目录存在
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # 构建输出文件路径
    output_file_path = os.path.join(output_dir, os.path.basename(file_path).replace('.csv', '_features.csv'))
    
    # 将特征保存到CSV文件中
    features_df.to_csv(output_file_path, index=False)

def process_all_files_in_directory(directory_path, output_dir, wavelet='db2', max_level=None):
    # 构建目录路径中所有CSV文件的搜索模式
    search_pattern = os.path.join(directory_path, "*.csv")
    
    # 使用glob找到所有匹配的文件
    csv_files = glob.glob(search_pattern)
    
    # 遍历所有找到的CSV文件
    for file_path in csv_files:
        # 对每个文件执行小波包分析并保存特征
        process_and_save_features(file_path, output_dir, wavelet, max_level)

# 设置源文件目录和目标目录路径
directory_path = "E:/毕设/电路仿真/仿真结果/four_op_amp/原始数据文件/R6+/"  # 源文件目录路径
output_dir = "E:/毕设/电路仿真/仿真结果/four_op_amp/小波包分解2/R6+/"  # 输出目录路径

# 处理目录下的所有文件
process_all_files_in_directory(directory_path, output_dir)

print('数据处理完成')

数据处理完成


In [13]:
import os
import glob
import pandas as pd
import pywt
import numpy as np

def process_and_save_features(file_path, output_dir, wavelet='db2', max_level=None):
    # 读取数据，跳过第一行（列名）
    df = pd.read_csv(file_path, header=None, skiprows=1)
    
    # 将第二列转换为浮点数，用作信号
    signal = df.iloc[:, 1].astype(float).values
    
    # 执行小波包分析
    wp = pywt.WaveletPacket(data=signal, wavelet=wavelet, mode='symmetric', maxlevel=max_level)
    
    # 提取特征：这里我们简单地使用每个节点的能量作为特征
    features = {node.path: np.sum(node.data**2) for node in wp.get_level(max_level or wp.maxlevel, 'freq')}
    
    # 将特征保存到DataFrame中
    features_df = pd.DataFrame([features])
    
    # 确保输出目录存在
    os.makedirs(output_dir, exist_ok=True)
    
    # 构建输出文件路径
    output_file_path = os.path.join(output_dir, os.path.basename(file_path).replace('.csv', '_features.csv'))
    
    # 将特征保存到CSV文件中
    features_df.to_csv(output_file_path, index=False)

def process_directory_recursively(source_directory, output_base_dir, wavelet='db2', max_level=None):
    for root, dirs, files in os.walk(source_directory):
        for dir_name in dirs:
            if dir_name == "all_data":
                continue
            
            source_sub_dir = os.path.join(root, dir_name)
            output_sub_dir = source_sub_dir.replace(source_directory, output_base_dir, 1)
            
            # 处理当前目录下的所有CSV文件
            process_all_files_in_directory(source_sub_dir, output_sub_dir, wavelet, max_level)

def process_all_files_in_directory(directory_path, output_dir, wavelet='db2', max_level=None):
    search_pattern = os.path.join(directory_path, "*.csv")
    csv_files = glob.glob(search_pattern)
    for file_path in csv_files:
        process_and_save_features(file_path, output_dir, wavelet, max_level)

# 设置源文件目录和目标目录的基路径
source_directory = "E:/毕设/电路仿真/仿真结果/four_op_amp/原始数据文件"  # 源文件基目录路径
output_base_dir = "E:/毕设/电路仿真/仿真结果/four_op_amp/小波包分解12"  # 输出基目录路径  8

# 处理源目录下的所有文件夹（除了 all_data）
process_directory_recursively(source_directory, output_base_dir,wavelet='sym3')

print('数据处理完成')


数据处理完成


In [4]:
import pywt

# 打印所有离散小波名称
print("Discrete Wavelets:", pywt.wavelist(kind='discrete'))

# 打印所有连续小波名称
print("Continuous Wavelets:", pywt.wavelist(kind='continuous'))

Discrete Wavelets: ['bior1.1', 'bior1.3', 'bior1.5', 'bior2.2', 'bior2.4', 'bior2.6', 'bior2.8', 'bior3.1', 'bior3.3', 'bior3.5', 'bior3.7', 'bior3.9', 'bior4.4', 'bior5.5', 'bior6.8', 'coif1', 'coif2', 'coif3', 'coif4', 'coif5', 'coif6', 'coif7', 'coif8', 'coif9', 'coif10', 'coif11', 'coif12', 'coif13', 'coif14', 'coif15', 'coif16', 'coif17', 'db1', 'db2', 'db3', 'db4', 'db5', 'db6', 'db7', 'db8', 'db9', 'db10', 'db11', 'db12', 'db13', 'db14', 'db15', 'db16', 'db17', 'db18', 'db19', 'db20', 'db21', 'db22', 'db23', 'db24', 'db25', 'db26', 'db27', 'db28', 'db29', 'db30', 'db31', 'db32', 'db33', 'db34', 'db35', 'db36', 'db37', 'db38', 'dmey', 'haar', 'rbio1.1', 'rbio1.3', 'rbio1.5', 'rbio2.2', 'rbio2.4', 'rbio2.6', 'rbio2.8', 'rbio3.1', 'rbio3.3', 'rbio3.5', 'rbio3.7', 'rbio3.9', 'rbio4.4', 'rbio5.5', 'rbio6.8', 'sym2', 'sym3', 'sym4', 'sym5', 'sym6', 'sym7', 'sym8', 'sym9', 'sym10', 'sym11', 'sym12', 'sym13', 'sym14', 'sym15', 'sym16', 'sym17', 'sym18', 'sym19', 'sym20']
Continuous Wave